## Entity Matching Pipeline with rule based matching 

Notebook showing the implementation of an EM pipeline with rule-based matching.
Attribute based blocking is done, along with three matching rules on the basis of features obtained from the similarity values of the attributes

## Dataset exploration 
After going through multiple datasets like movies.csv; citations.csv etc., we chose the music dataset. 
In other datasets, we had issues like inaccessibility, same attributes not being present across the two datasets etc. 
The two music datasets have similar attributes across them. Attributes like Genre etc. can also be used as sensitive attributes.
The labeled dataset of size 539 was also available for music datasets for evaluation purpose.

In [105]:
# Importing the required libraries
import sys
import py_entitymatching as em
import pandas as pd
import os

In [106]:
# Load csv files as dataframes and set the key attribute in the dataframe
path_A = 'music1.csv'
path_B = 'music2.csv'
A = em.read_csv_metadata(path_A, key='Sno')
B = em.read_csv_metadata(path_B, key='Sno')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [107]:
# Display number of tuples in the datasets
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 6907
Number of tuples in B: 55923
Number of tuples in A X B (i.e the cartesian product): 386260161


In [108]:
# Displaying first two entries from the first music dataset
A.head(2)

,Sno,Album_Name,Album_Price,Artist_Name,CopyRight,Customer_Rating,Genre,Price,Released,Song_Name,Time,Group
0,1,Welcome to Cam Country - EP,$4.26,Cam,2015 Sony Music Entertainment,4.72396,"Country,Music,Contemporary Country,Honky Tonk",$0.99,31-Mar-15,Runaway Train,3:01,Country
1,2,Me 4 U,$9.99,Omi,"2015 Ultra Records, LLC under exclusive license to Columbia Records, a Division of Sony Music E...",3.38158,"Pop/Rock,Music,Pop,Dance,R&B/Soul",Album Only,NaN,Track 14,3:41,Pop


In [109]:
# Displaying first two entries from the second music dataset
B.head(2)

,Sno,Album_Name,Artist_Name,Song_Name,Price,Time,Released,Label,Copyright,Genre,Group
0,1,! (Volume 2) [Explicit],Rusko,Saxophone Stomp [Explicit],$1.29,3:20,"September 16, 2014",Decca International,(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"Dance & Electronic,Dubstep",Dance
1,2,! (Volume 2) [Explicit],Rusko,I Wanna Mingle [feat. Pusher],$1.29,2:36,"September 16, 2014",Decca International,(C) 2014 FMLY Under Exclusive License To Universal Music Canada Inc.,"Dance & Electronic,Dubstep",Dance


In [110]:
# Display the keys of the input tables
print(em.get_key(A), em.get_key(B))
# If the tables are large we can downsample the tables like this
A1, B1 = em.down_sample(A, B, 1000, 1, show_progress=False)
print("Lengths after downsampling-")
print(len(A1), len(B1))

Sno Sno
Lengths after downsampling-
572 1000


Blocking

In [111]:
# Create attribute equivalence blocker
ab = em.AttrEquivalenceBlocker()

# Block using artist_name attribute
C = ab.block_tables(A1, B1, "Artist_Name","Artist_Name",
                    l_output_attrs=["Sno", "Album_Name", "Artist_Name", "CopyRight", "Released", "Song_Name" ,"Time"], 
                    r_output_attrs=["Sno", "Album_Name", "Artist_Name", "Copyright", "Released", "Song_Name", "Time"]
                    )

# Rule based blocker, which blocks on the basis of a rule
rb = em.RuleBasedBlocker()
block_f = em.get_features_for_blocking(A1, B1,validate_inferred_attr_types=False)
rule1 = ["Artist_Name_Artist_Name_lev_sim(ltuple, rtuple) > 0.8"]
rb.add_rule(rule1, block_f)
C = rb.block_tables(A1, B1, 
                    l_output_attrs=["Sno", "Album_Name", "Artist_Name", "CopyRight", "Released", "Song_Name" ,"Time"], 
                    r_output_attrs=["Sno", "Album_Name", "Artist_Name", "Copyright", "Released", "Song_Name", "Time"]
                    )

In [112]:
# Printing length of candidate set
len(C)

1482

Matching

In [9]:
# Sample candidate set
# S = em.sample_table(C, 538)

In [10]:
# Label S
# G = em.label_table(S, 'label')

In [113]:
# Load the pre-labeled data
path_G = 'music_labeled_data.csv'
G = em.read_csv_metadata(path_G, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable.Sno', fk_rtable='rtable.Sno')
print(len(G))

Metadata file is not present in the given path; proceeding to read the csv file.


539


In [115]:
# Generate features
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

In [116]:
G.head(2)

,Unnamed: 0,_id,ltable.Sno,rtable.Sno,ltable.Album_Name,ltable.Artist_Name,ltable.CopyRight,ltable.Released,ltable.Song_Name,ltable.Time,rtable.Album_Name,rtable.Artist_Name,rtable.CopyRight,rtable.Released,rtable.Song_Name,rtable.Time,label,ltable.genre,rtable.genre
0,916,916,111,53124,vhs,x ambassadors,2015 kidinakorner/interscope records,30-Jun-15,vhs outro (interlude),1:25,vhs [explicit],x ambassadors,(c) 2015 kidinakorner/interscope records,"June 30, 2015",vhs outro (interlude) [explicit],1:25,1,Alternative,Alternative
1,1053,1053,148,50767,title (deluxe),meghan trainor,"2014, 2015 epic records, a division of sony music entertainment",9-Jan-15,credit,2:51,title (deluxe),meghan trainor,"2011 what a music ltd, licence exclusive parlophone music france","January 9, 2015",credit,2:51,1,Pop,Pop


In [117]:
# Listing the names of the features generated
feature_table['feature_name']

0                                     Sno_Sno_exm
1                                     Sno_Sno_anm
2                                Sno_Sno_lev_dist
3                                 Sno_Sno_lev_sim
4           Album_Name_Album_Name_jac_qgm_3_qgm_3
5       Album_Name_Album_Name_cos_dlm_dc0_dlm_dc0
6       Album_Name_Album_Name_jac_dlm_dc0_dlm_dc0
7                       Album_Name_Album_Name_mel
8                  Album_Name_Album_Name_lev_dist
9                   Album_Name_Album_Name_lev_sim
10                      Album_Name_Album_Name_nmw
11                       Album_Name_Album_Name_sw
12        Artist_Name_Artist_Name_jac_qgm_3_qgm_3
13    Artist_Name_Artist_Name_cos_dlm_dc0_dlm_dc0
14    Artist_Name_Artist_Name_jac_dlm_dc0_dlm_dc0
15                    Artist_Name_Artist_Name_mel
16               Artist_Name_Artist_Name_lev_dist
17                Artist_Name_Artist_Name_lev_sim
18                    Artist_Name_Artist_Name_nmw
19                     Artist_Name_Artist_Name_sw


In [118]:
# Manually creating features for some attribute

def time_feature(ltuple, rtuple):
    p1 = ltuple.Time
    p2 = rtuple.Time
    if p1 == p2:
        return 1.0
    else:
        return 0.0
    
def released_feature(ltuple, rtuple):
    p1 = str(ltuple.Released)
    p2 = str(rtuple.Released)
    if p1=="nan" or p2=="nan":
        return 0
    p1 = p1.split("-") # 31,mar,15
    if len(p1)<3:
        return 0
    d = {"Jan":"January","Feb":"February","Mar":"March","Apr":"April","May":"May","Jun":"June","Jul":"July","Aug":"August","Sep":"September","Oct":"October","Nov":"November","Dec":"December"}
    s1 = " " + d[p1[1]] + " " + p1[0] + ", 20" + p1[2]
    return em.lev_sim(s1,p2)
    
def copyright_feature(ltuple, rtuple):
    p1 = ltuple.CopyRight
    p2 = rtuple.Copyright
    
    return em.lev_sim(p1,p2)

def price_feature(ltuple, rtuple):
    p1 = str(ltuple.Price).strip()
    p2 = str(rtuple.Price).strip()
    if p1 == p2:
        return 1.0
    else:
        return 0.0

In [119]:
em.add_blackbox_feature(feature_table, 'time_time_feature', time_feature)
em.add_blackbox_feature(feature_table, 'released_released_feature', released_feature)
em.add_blackbox_feature(feature_table, 'price_price_feature', price_feature)
em.add_blackbox_feature(feature_table, 'copyright_copyright_feature', copyright_feature)

H = em.extract_feature_vecs(G, feature_table=feature_table)
H

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


,_id,ltable.Sno,rtable.Sno,Sno_Sno_exm,Sno_Sno_anm,Sno_Sno_lev_dist,Sno_Sno_lev_sim,Album_Name_Album_Name_jac_qgm_3_qgm_3,Album_Name_Album_Name_cos_dlm_dc0_dlm_dc0,Album_Name_Album_Name_jac_dlm_dc0_dlm_dc0,...,Group_Group_lev_dist,Group_Group_lev_sim,Group_Group_jar,Group_Group_jwn,Group_Group_exm,Group_Group_jac_qgm_3_qgm_3,time_time_feature,released_released_feature,price_price_feature,copyright_copyright_feature
0,916,111,53124,0,0.002089,4,0.2,0.166667,0.707107,0.500000,...,0.0,1.0,1.0,1.0,1,1.0,1.0,1.000000,1.0,0.902439
1,1053,148,50767,0,0.002915,5,0.0,1.000000,1.000000,1.000000,...,0.0,1.0,1.0,1.0,1,1.0,1.0,1.000000,1.0,0.276923
2,1290,206,41214,0,0.004998,4,0.2,0.600000,0.666667,0.500000,...,0.0,1.0,1.0,1.0,1,1.0,1.0,1.000000,1.0,0.878788
3,1424,211,19812,0,0.010650,4,0.2,0.150943,0.000000,0.000000,...,5.0,0.0,0.0,0.0,0,0.0,0.0,0.333333,1.0,0.310345
4,1706,250,53111,0,0.004707,5,0.0,0.166667,0.707107,0.500000,...,0.0,1.0,1.0,1.0,1,1.0,0.0,1.000000,1.0,0.902439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,57960,6867,13598,0,0.505001,5,0.0,0.480000,0.707107,0.500000,...,0.0,1.0,1.0,1.0,1,1.0,0.0,1.000000,1.0,0.131313
535,57993,6868,13597,0,0.505111,5,0.0,0.480000,0.707107,0.500000,...,0.0,1.0,1.0,1.0,1,1.0,0.0,1.000000,1.0,0.131313
536,58289,6877,13623,0,0.504808,5,0.0,0.444444,0.707107,0.500000,...,0.0,1.0,1.0,1.0,1,1.0,0.0,1.000000,1.0,0.131313
537,58403,6882,13611,0,0.505620,5,0.0,0.444444,0.707107,0.500000,...,0.0,1.0,1.0,1.0,1,1.0,0.0,1.000000,1.0,0.131313


In [61]:
result_list = []

In [158]:
# Initializing the rule based matcher
brm = em.BooleanRuleMatcher()

# Add four rules to the rule-based matcher

# The first rule compares the album names
brm.add_rule(['Album_Name_Album_Name_lev_sim(ltuple, rtuple) > 0.95'], feature_table)

# This second rule compares the genres
brm.add_rule(['Genre_Genre_lev_sim(ltuple, rtuple) > 0.8'], feature_table)

# This third rule compares the times
brm.add_rule(['time_time_feature(ltuple, rtuple) == 1.0'],feature_table)

# This fourth rule compares the song name
brm.add_rule(['Song_Name_Song_Name_lev_sim(ltuple, rtuple) > 0.9'],feature_table)



#Additional not to be used yet

# brm.add_rule(['released_released_feature(ltuple, rtuple) > 0.9'],feature_table)
# brm.add_rule(['copyright_copyright_feature(ltuple, rtuple) == 1.0'],feature_table)
# brm.add_rule(['price_price_feature(ltuple, rtuple) == 1.0'],feature_table)

print(brm.get_rule_names())

brm.predict(G, target_attr='pred_label', append=True)
G.head(2)

odict_keys(['_rule_0', '_rule_1', '_rule_2', '_rule_3'])


,Unnamed: 0,_id,ltable.Sno,rtable.Sno,ltable.Album_Name,ltable.Artist_Name,ltable.CopyRight,ltable.Released,ltable.Song_Name,ltable.Time,rtable.Album_Name,rtable.Artist_Name,rtable.CopyRight,rtable.Released,rtable.Song_Name,rtable.Time,label,ltable.genre,rtable.genre,pred_label
0,916,916,111,53124,vhs,x ambassadors,2015 kidinakorner/interscope records,30-Jun-15,vhs outro (interlude),1:25,vhs [explicit],x ambassadors,(c) 2015 kidinakorner/interscope records,"June 30, 2015",vhs outro (interlude) [explicit],1:25,1,Alternative,Alternative,1
1,1053,1053,148,50767,title (deluxe),meghan trainor,"2014, 2015 epic records, a division of sony music entertainment",9-Jan-15,credit,2:51,title (deluxe),meghan trainor,"2011 what a music ltd, licence exclusive parlophone music france","January 9, 2015",credit,2:51,1,Pop,Pop,1


In [159]:
# Evaluate the predictions
eval_result = em.eval_matches(G, 'label', 'pred_label')
em.print_eval_summary(eval_result)

Precision : 78.26% (126/161)
Recall : 95.45% (126/132)
F1 : 86.01%
False positives : 35 (out of 161 positive predictions)
False negatives : 6 (out of 378 negative predictions)


Fairness Vector

In [160]:
#get fairness vector metrics

H['pred_label'] = G['pred_label']
pred_match = H[H['pred_label'] ==1] 


fairness_names = ['Genre_Genre_lev_sim','time_time_feature','Song_Name_Song_Name_lev_sim','Album_Name_Album_Name_lev_sim']
additional = ['released_released_feature','copyright_copyright_feature','price_price_feature']

fair_vec = []

# print(pred_match.shape)
# print(H.shape)

for f in fairness_names:
    fair_vec.append(sum(pred_match[f]))

for f in additional:
    fair_vec.append(sum(pred_match[f]))


fair_vec = np.array(fair_vec)

# print("Fainress Vector",fair_vec)
res = [len(brm.get_rule_names()),eval_result['precision'],eval_result['recall'],eval_result['f1'], fair_vec.std()]
print("Standard Deviation = ", fair_vec.std())

result_list.append(res)

Standard Deviation =  38.35533250289443


In [161]:
from tabulate import tabulate
print(tabulate(result_list,headers=['Length', 'Precision','Recall','F1','Std. Dev']))

  Length    Precision    Recall        F1    Std. Dev
--------  -----------  --------  --------  ----------
       4     0.782609  0.954545  0.860068     38.3553
       3     0.734848  0.734848  0.734848     33.5544
       2     0.611111  0.416667  0.495495     25.5018
       2     1         0.924242  0.96063      30.3824
       3     0.782609  0.954545  0.860068     38.3553
       1     1         0.651515  0.788991     22.579
       4     0.782609  0.954545  0.860068     38.3553


# For involving fairness constraints, we take Genre as the sensitive attributes
#### The following distribution of distinct categories was obtained for the two datasets-->
#Music 1 : {'Pop': 1758, 'Hip-Hop': 685, 'Rock': 515, 'Dance': 952, 'Country': 1367, 
'Electronic': 211, 'Alternative': 484, 'Soundtrack': 397, 'Other': 538}

#Music 2 : {'Pop': 7130, 'Rap & Hip-Hop': 3400, 'Rock': 3248, 'Dance': 8274, 'Country': 8317, 'Electronic': 4019, 'Alternative': 4806, 'Soundtrack': 3594, 'Other': 13135}

Rap & Hip-Hop converted to Hip-Hop in music2 for consistency.

## There were mamy distinct categories in the dataset, they have been clubbed into umbrella categories as shown above.


In [19]:
#In case of matching and different groups in music1 and music2, group of music1 is taken (15 such cases were there)
NG = G.to_numpy()

mgs = list()
lst = ['Pop','Hip-Hop','Rock','Dance','Country','Electronic','Alternative','Soundtrack','Other']
for z in lst:
    cnt,pos,neg,fp,fn= [0,0,0,0,0]
    for x in NG:
        if(x[-2]== z):
            cnt+=1
            if(x[-1]==1 and x[-4]==1):
                pos+=1
            elif(x[-1]==1 and x[-4]==0):
                fp+=1
            elif(x[-1]==0 and x[-4]==1):
                fn+=1
            else:
                neg+=1
    print('Total count of '+z+' = '+str(cnt))
    print('Positive = ',pos)
    print('Negative = ',neg)
    print('False Pos = ',fp)
    print('False Neg = ',fn)
    mg_ = (fp+fn)/cnt
    print('Mg = ', mg_)
    mgs.append(mg_)
    print('\n\n\n')


Total count of Pop = 85
Positive =  30
Negative =  51
False Pos =  4
False Neg =  0
Mg =  0.047058823529411764




Total count of Hip-Hop = 121
Positive =  17
Negative =  101
False Pos =  0
False Neg =  3
Mg =  0.024793388429752067




Total count of Rock = 34
Positive =  7
Negative =  27
False Pos =  0
False Neg =  0
Mg =  0.0




Total count of Dance = 156
Positive =  17
Negative =  116
False Pos =  22
False Neg =  1
Mg =  0.14743589743589744




Total count of Country = 57
Positive =  38
Negative =  15
False Pos =  4
False Neg =  0
Mg =  0.07017543859649122




Total count of Electronic = 27
Positive =  3
Negative =  21
False Pos =  3
False Neg =  0
Mg =  0.1111111111111111




Total count of Alternative = 27
Positive =  9
Negative =  17
False Pos =  0
False Neg =  1
Mg =  0.037037037037037035




Total count of Soundtrack = 4
Positive =  1
Negative =  1
False Pos =  2
False Neg =  0
Mg =  0.5




Total count of Other = 28
Positive =  4
Negative =  23
False Pos =  0
False Neg =  1
M

In [20]:
gmu = sum(mgs)/len(mgs)
print('GMU = ',gmu)
biases=list()
for i,x in enumerate(lst):
    bias = (gmu - mgs[i])/gmu
    biases.append(bias)
    print('Total bias in '+x+' = '+str(bias))

GMU =  0.10814733131710959
Total bias in Pop = 0.5648637561714224
Total bias in Hip-Hop = 0.7707443343465139
Total bias in Rock = 1.0
Total bias in Dance = -0.36328743058472635
Total bias in Country = 0.35111261885212114
Total bias in Electronic = -0.02740502015080818
Total bias in Alternative = 0.6575316599497306
Total bias in Soundtrack = -3.623322590678637
Total bias in Other = 0.6697626720943831


#### "If G’s mismatches are equal to the average number of mismatches of all groups, then its bias is equal to zero. On the other hand, if the mismatches of G are more than the average, then bias gets a negative value, and, in this case, we consider that the method examined is negatively biased against G. 

#### Finally, when the average number of mismatches is greater than that of G, then we assume that the method is biased in favor ofG. Values close to 0 indicate low bias, while as the absolute value of bias increases, this is an indication of increased bias, either positive or negative." - EDBT PAPER

In [21]:
import numpy as np 
import matplotlib.pyplot as plt  
     
fig = plt.figure(figsize = (10, 5))

lst.append('AVERAGE')
biases.append(gmu)

plt.bar(lst, biases, color ='blue', width = 0.4) 
  
plt.xlabel("Genre Group") 
plt.ylabel("Bias") 
plt.title("Bias in Groups") 
plt.show() 

<Figure size 1000x500 with 1 Axes>